In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(project_root)

In [2]:
from datasets import load_dataset

sst2 = load_dataset("stanfordnlp/sst2")
class_dataset = sst2['train']['sentence'][:100]
class_targets = sst2['train']['label'][:100]

samsum = load_dataset("knkarthick/samsum")
gen_dataset = samsum['train']['dialogue'][:100]
gen_targets = samsum['train']['summary'][:100]

Starting with PromptTuner

In [3]:
from coolprompt.assistant import PromptTuner

# Define an initial prompt
class_start_prompt = "Classify sentence sentiment"

# Initialize the tuner
tuner = PromptTuner()

# Call prompt optimization with dataset and target
final_prompt = tuner.run(
    start_prompt=class_start_prompt,
    task="classification",
    dataset=class_dataset,
    target=class_targets,
    metric="f1"
)

[2025-07-09 00:56:30,551] [INFO] [llm.init] - Initializing default model
[2025-07-09 00:56:30,551] [DEBUG] [llm.init] - Updating default model params with langchain config: None and vllm_engine_config: None


INFO 07-09 00:56:32 __init__.py:207] Automatically detected platform cuda.
WARNING 07-09 00:56:32 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 07-09 00:56:40 config.py:549] This model supports multiple tasks: {'classify', 'reward', 'generate', 'embed', 'score'}. Defaulting to 'generate'.
INFO 07-09 00:56:40 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='t-tech/T-lite-it-1.0', speculative_config=None, tokenizer='t-tech/T-lite-it-1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-09 00:58:48 model_runner.py:1115] Loading model weights took 14.2426 GB
INFO 07-09 00:58:53 worker.py:267] Memory profiling takes 5.28 seconds
INFO 07-09 00:58:53 worker.py:267] the current vLLM instance can use total_gpu_memory (23.64GiB) x gpu_memory_utilization (0.90) = 21.27GiB
INFO 07-09 00:58:53 worker.py:267] model weights take 14.24GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 4.35GiB; the rest of the memory reserved for KV Cache is 2.62GiB.
INFO 07-09 00:58:54 executor_base.py:111] # cuda blocks: 3069, # CPU blocks: 4681
INFO 07-09 00:58:54 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 1.50x
INFO 07-09 00:59:01 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████████████████████| 35/35 [00:16<00:00,  2.11it/s]

INFO 07-09 00:59:18 model_runner.py:1562] Graph capturing finished in 17 secs, took 0.21 GiB
INFO 07-09 00:59:18 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 30.00 seconds



[2025-07-09 00:59:18,449] [INFO] [assistant.__init__] - Validating the model: VLLM
[2025-07-09 00:59:18,451] [INFO] [assistant.__init__] - PromptTuner successfully initialized with model: VLLM
[2025-07-09 00:59:18,452] [INFO] [assistant.run] - Validating args for PromptTuner running
[2025-07-09 00:59:19,205] [INFO] [evaluator.__init__] - Evaluator sucessfully initialized with f1 metric
[2025-07-09 00:59:19,206] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-07-09 00:59:19,206] [INFO] [assistant.run] - Method: hype, Task: classification
[2025-07-09 00:59:19,207] [INFO] [assistant.run] - Metric: f1, Validation size: 0.25
[2025-07-09 00:59:19,208] [INFO] [assistant.run] - Dataset: 100 samples
[2025-07-09 00:59:19,208] [INFO] [assistant.run] - Target: 100 samples
[2025-07-09 00:59:19,209] [INFO] [hype.hype_optimizer] - Running HyPE optimization...
Processed prompts: 100%|█| 1/1 [00:00<00:00,  1.32it/s, est. speed input: 328.07 toks/s, out
[2025-07-09 00:59:19,993] [IN

In [4]:
print("Final prompt:", final_prompt)
print("Start prompt metric: ", tuner.init_metric)
print("Final prompt metric: ", tuner.final_metric)

Final prompt: Please provide a sentence and classify its sentiment as positive, negative, or neutral.
Start prompt metric:  0.6364983164983165
Final prompt metric:  0.8899889988998899


You can do the same with generation task

Also you can reuse previous tuner binded to LLM

In [3]:
from coolprompt.assistant import PromptTuner

# Define an initial prompt
class_start_prompt = "Classify sentence sentiment"

# Initialize the tuner
tuner = PromptTuner()

[2025-07-09 01:04:23,098] [INFO] [llm.init] - Initializing default model
[2025-07-09 01:04:23,099] [DEBUG] [llm.init] - Updating default model params with langchain config: None and vllm_engine_config: None


INFO 07-09 01:04:24 __init__.py:207] Automatically detected platform cuda.
WARNING 07-09 01:04:25 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 07-09 01:04:33 config.py:549] This model supports multiple tasks: {'classify', 'embed', 'score', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 07-09 01:04:33 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='t-tech/T-lite-it-1.0', speculative_config=None, tokenizer='t-tech/T-lite-it-1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-09 01:06:41 model_runner.py:1115] Loading model weights took 14.2426 GB
INFO 07-09 01:06:46 worker.py:267] Memory profiling takes 5.32 seconds
INFO 07-09 01:06:46 worker.py:267] the current vLLM instance can use total_gpu_memory (23.64GiB) x gpu_memory_utilization (0.90) = 21.27GiB
INFO 07-09 01:06:46 worker.py:267] model weights take 14.24GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 4.35GiB; the rest of the memory reserved for KV Cache is 2.62GiB.
INFO 07-09 01:06:47 executor_base.py:111] # cuda blocks: 3069, # CPU blocks: 4681
INFO 07-09 01:06:47 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 1.50x
INFO 07-09 01:06:54 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████████████████████| 35/35 [00:16<00:00,  2.07it/s]

INFO 07-09 01:07:11 model_runner.py:1562] Graph capturing finished in 17 secs, took 0.21 GiB
INFO 07-09 01:07:11 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 30.44 seconds



[2025-07-09 01:07:11,872] [INFO] [assistant.__init__] - Validating the model: VLLM
[2025-07-09 01:07:11,873] [INFO] [assistant.__init__] - PromptTuner successfully initialized with model: VLLM


In [4]:
gen_start_prompt = "Summarize this text"

final_prompt = tuner.run(
    start_prompt=gen_start_prompt,
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
    metric="meteor"
)

print("Final prompt:", final_prompt)
print("Start prompt metric: ", tuner.init_metric)
print("Final prompt metric: ", tuner.final_metric)

[2025-07-09 01:09:57,414] [INFO] [assistant.run] - Validating args for PromptTuner running
[nltk_data] Downloading package wordnet to
[nltk_data]     /nfs/home/asitkina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /nfs/home/asitkina/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /nfs/home/asitkina/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[2025-07-09 01:09:59,450] [INFO] [evaluator.__init__] - Evaluator sucessfully initialized with meteor metric
[2025-07-09 01:09:59,450] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-07-09 01:09:59,451] [INFO] [assistant.run] - Method: hype, Task: generation
[2025-07-09 01:09:59,451] [INFO] [assistant.run] - Metric: meteor, Validation size: 0.25
[2025-07-09 01:09:59,452] [INFO] [assistant.run] - Dataset: 100 samples
[2025-07-09 01:09:59,453] [INFO] 

Final prompt: Please provide a concise summary of the text, focusing on the main points and omitting any unnecessary details.
Start prompt metric:  0.2889385547946554
Final prompt metric:  0.30479966549868787


Currently supported metrics are
- accuracy and f1 for classification
- meteor, bleu and rouge for generation

Also, task type must correspond the metric

In [5]:
tuner.run(
    start_prompt=class_start_prompt,
    task="classification",
    dataset=class_dataset,
    target=class_targets,
    metric="rouge"
)

[2025-07-09 01:10:35,570] [INFO] [assistant.run] - Validating args for PromptTuner running
[2025-07-09 01:10:35,571] [ERROR] [metrics.validate_and_create_metric] - Invalid metric for classification task: rouge. Available metrics: accuracy, f1.


ValueError: Invalid metric for classification task: rouge. Available metrics: accuracy, f1.

There are two ways to initialize tuner with your custom LLM

To init a model by yourself and pass it to the tuner

In [1]:
from langchain_community.llms import VLLM

my_model = VLLM(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    trust_remote_code=True,
    dtype='float16',
)

tuner_with_custom_llm = PromptTuner(model=my_model)
tuner_with_custom_llm.run(start_prompt="Write an essay about autumn")

KeyboardInterrupt: 

Or to change config of our default model

In [7]:
from coolprompt.language_model.llm import DefaultLLM

changed_model = DefaultLLM.init(langchain_config={
    'max_new_tokens': 1000,
    "temperature": 0.0,
})

tuner_with_changed_llm = PromptTuner(model=changed_model)
tuner_with_changed_llm.run(start_prompt="Write an essay about autumn")

[2025-07-09 01:02:04,899] [INFO] [llm.init] - Initializing default model


WARNING 07-09 01:02:05 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 07-09 01:02:05 config.py:549] This model supports multiple tasks: {'classify', 'reward', 'generate', 'embed', 'score'}. Defaulting to 'generate'.
INFO 07-09 01:02:05 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='t-tech/T-lite-it-1.0', speculative_config=None, tokenizer='t-tech/T-lite-it-1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 23.64 GiB of which 11.56 MiB is free. Including non-PyTorch memory, this process has 23.61 GiB memory in use. Of the allocated memory 23.17 GiB is allocated by PyTorch, with 46.00 MiB allocated in private pools (e.g., CUDA Graphs), and 81.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

You can access prompts and their metrics via tuner fields

In [7]:
def print_results(tuner):
    print("Start prompt:", tuner.init_prompt)
    print("Final prompt:", tuner.final_prompt)
    print("Start prompt metric: ", tuner.init_metric)
    print("Final prompt metric: ", tuner.final_metric)

print_results(tuner)

Start prompt: Summarize this text
Final prompt: Please provide a concise summary of the text, focusing on the main points and omitting any unnecessary details.
Start prompt metric:  0.2889385547946554
Final prompt metric:  0.30479966549868787


There are 3 currently implemented optimizers:
- HyPE (optimizing with predefined system instruction)
- DistillPrompt
- ReflectivePrompt

In [8]:
tuner.run(
    start_prompt=class_start_prompt,
    task="classification",
    method="hype",
    dataset=class_dataset,
    target=class_targets,
    metric="f1"
)

print_results(tuner)

[2025-07-09 01:11:35,596] [INFO] [assistant.run] - Validating args for PromptTuner running
[2025-07-09 01:11:36,329] [INFO] [evaluator.__init__] - Evaluator sucessfully initialized with f1 metric
[2025-07-09 01:11:36,330] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-07-09 01:11:36,330] [INFO] [assistant.run] - Method: hype, Task: classification
[2025-07-09 01:11:36,331] [INFO] [assistant.run] - Metric: f1, Validation size: 0.25
[2025-07-09 01:11:36,332] [INFO] [assistant.run] - Dataset: 100 samples
[2025-07-09 01:11:36,332] [INFO] [assistant.run] - Target: 100 samples
[2025-07-09 01:11:36,333] [INFO] [hype.hype_optimizer] - Running HyPE optimization...
Processed prompts: 100%|█| 1/1 [00:00<00:00,  1.29it/s, est. speed input: 320.47 toks/s, out
[2025-07-09 01:11:37,114] [INFO] [hype.hype_optimizer] - HyPE optimization completed
[2025-07-09 01:11:37,115] [INFO] [assistant.run] - Evaluating on given dataset for classification task...
[2025-07-09 01:11:37,115] [INFO]

Start prompt: Classify sentence sentiment
Final prompt: Please provide a sentence and classify its sentiment as positive, negative, or neutral.
Start prompt metric:  0.6364983164983165
Final prompt metric:  0.8899889988998899


In [9]:
tuner.run(
    start_prompt=class_start_prompt,
    task="classification",
    dataset=class_dataset[:10],
    target=class_targets[:10],
    method="distill",
    use_cache=False,
    num_epochs=1
)

print_results(tuner)

[2025-07-09 01:13:32,056] [INFO] [assistant.run] - Validating args for PromptTuner running
[2025-07-09 01:13:32,794] [INFO] [evaluator.__init__] - Evaluator sucessfully initialized with f1 metric
[2025-07-09 01:13:32,795] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-07-09 01:13:32,795] [INFO] [assistant.run] - Method: distill, Task: classification
[2025-07-09 01:13:32,796] [INFO] [assistant.run] - Metric: f1, Validation size: 0.25
[2025-07-09 01:13:32,796] [INFO] [assistant.run] - Dataset: 10 samples
[2025-07-09 01:13:32,797] [INFO] [assistant.run] - Target: 10 samples
[2025-07-09 01:13:32,809] [INFO] [distiller.distillation] - Starting DistillPrompt optimization...
[2025-07-09 01:13:32,810] [INFO] [evaluator.evaluate] - Evaluating prompt for classification task on 7 samples
Processed prompts: 100%|█| 7/7 [01:51<00:00, 15.91s/it, est. speed input: 2.96 toks/s, outpu
  0%|                                                                 | 0/1 [00:00<?, ?it/s][2025-

Start prompt: Classify sentence sentiment
Final prompt: Classify the sentiment of a sentence as positive, negative, or neutral.
Examples:
- "I love this movie!" is positive.
- "This movie is terrible." is negative.
- "The movie was okay." is neutral.
Start prompt metric:  0.23076923076923078
Final prompt metric:  1.0


In [ ]:
tuner.run(
    start_prompt=class_start_prompt,
    task="classification",
    dataset=class_dataset[:10],
    target=class_targets[:10],
    method="reflective",
    problem_description="sentiment classification",
    use_cache=False,
    population_size=4,
    num_epochs=1
)

print_results(tuner)

[2025-07-09 01:35:54,509] [INFO] [assistant.run] - Validating args for PromptTuner running
[2025-07-09 01:35:55,246] [INFO] [evaluator.__init__] - Evaluator sucessfully initialized with f1 metric
[2025-07-09 01:35:55,246] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-07-09 01:35:55,247] [INFO] [assistant.run] - Method: reflective, Task: classification
[2025-07-09 01:35:55,247] [INFO] [assistant.run] - Metric: f1, Validation size: 0.25
[2025-07-09 01:35:55,248] [INFO] [assistant.run] - Dataset: 10 samples
[2025-07-09 01:35:55,248] [INFO] [assistant.run] - Target: 10 samples
[2025-07-09 01:35:55,249] [INFO] [run.reflectiveprompt] - Starting ReflectivePrompt optimization...
[2025-07-09 01:35:55,250] [INFO] [evoluter._init_pop] - Initializing population...
Processed prompts: 100%|█| 1/1 [00:01<00:00,  1.78s/it, est. speed input: 44.52 toks/s, outp
[2025-07-09 01:35:57,029] [INFO] [evoluter._evaluation] - Evaluating population...
[2025-07-09 01:35:57,030] [INFO] [evalu

KeyboardInterrupt: 